In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
# Importing the dataset containing the tweets, the same dataste which was outputted by the last cell of Twitter app.
data = spark.read.format("csv").option("header","true").load("test1.csv") 

In [4]:
# Converting the dataset into rdd
data_rdd = data.rdd

In [181]:
# Removing the column name associated with the tweets
data_rdd_tweet_only = data_rdd.map(lambda x: x[0])

In [182]:
data_rdd_tweet_only.first()

'RT @BarbilCS: Poor &amp; worst medical #FacilitiesManagement @DistAdmKeonjhar exposed in minerals rich #ActualMiningAffected @Jodamhud @DDMJODA…'

In [50]:
# Defining the list of punctuations we will be removing before calculating the sentiment score
punctuations = ["(", "[", ",", ".", "!", "?", ":", ";", "]", ")", "@", "^",'#','&']

### Writing a function to replace punctuations in text with " "

In [51]:
def eliminate_punctuations(text):
    for char in text:
        if char in punctuations:
          text = text.replace(char,"")
    return text

In [177]:
# removing the punctuations and converting the entire tweet into lower case.
clean = data_rdd_pair.map(lambda text : eliminate_punctuations(text)).map(lambda text : text.lower())

In [178]:
clean.first()

'rt barbilcs poor amp worst medical facilitiesmanagement distadmkeonjhar exposed in minerals rich actualminingaffected jodamhud ddmjoda…'

In [140]:
# Importing text file containing the positive words
positive = spark.sparkContext.textFile('positive-words.txt')

In [141]:
# Importing text file containing the negative words
negative = spark.sparkContext.textFile('negative-words.txt')

In [142]:
# Creating a key value pair rdd with key as the tweet and value as the words in the tweet.
clean_pair = clean.map(lambda x : (x,x.split()))

In [143]:
clean_pair.first()

('rt barbilcs poor amp worst medical facilitiesmanagement distadmkeonjhar exposed in minerals rich actualminingaffected jodamhud ddmjoda…',
 ['rt',
  'barbilcs',
  'poor',
  'amp',
  'worst',
  'medical',
  'facilitiesmanagement',
  'distadmkeonjhar',
  'exposed',
  'in',
  'minerals',
  'rich',
  'actualminingaffected',
  'jodamhud',
  'ddmjoda…'])

In [183]:
# Extracting positive words in one list
positive_list = positive.collect()
# Extracting negative words in another list
negative_list = negative.collect()

### Writing a function which calculate sentiment score of the tweet. 
#### In this, each word of tweet is checked whether it is present in the positive  or negative list and accordingly 
#### the counter is incremented. In the end the sentiment score is calculated as (positive count - negative 
#### count 

In [144]:
def sentiment_score(x):
    positive_count = 0
    negative_count = 0
    for words in x:
        if words in positive_list:
            positive_count = positive_count + 1 
        elif words in negative_list:
            negative_count = negative_count + 1
    return(positive_count - negative_count)   

In [152]:
ans = clean_pair.map(lambda x : (x[0],sentiment_score(x[1])))

In [155]:
# Collecting the tweet and the sentiment score into the list.
lis = ans.collect()

In [156]:
import pandas as pd   
# Converting list to Dataframe
df = pd.DataFrame(lis) 

In [157]:
df.columns = (['Tweet_content','Sentiment Score'])

In [162]:
# Outputting the dataframe as a csv
df.to_csv('result.csv',encoding = 'utf-8',index = False) # Writing the final output to csv